# Lab 2B: Functional programming, and declarative patterns


__Student:__ aleka769 (Alexander Karlsson)

__Student:__ maxpf364 (Maximilian Pfundstein)

Disclaimer: Functional programming in Python does not always lead to the fastest possible code, and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model which many big data systems, e.g. Hadoop, relies on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigm language, and functional programming is one of the main paradigms one can use. To understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements unless this is explicitly allowed in the task.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together). Incidentally, one might find speedups when using the immutable tuples instead of lists.

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

# 2 Recursion

As an introduction to linear recursion, read the introductory note on the course webpage. This might help explain terms that you may not know (even if the concept is previously known).

## 2.1 Linear recursion

a) Write a recursive function `sum_even(n)` that takes a natural number $n\geq 0$ and returns the sum of all even numbers $0,...,n$. It should be linear-recursive with delayed computations.

In [1]:
def sum_even(n):
    if n == 1:
        return 1
    return n + sum_even(n-1)

print(sum_even(6))

21


b) Write `sum_even_it(n)` according to the same specification. In this case, the solution should be tail recursive.

In [2]:
# Your code here.
def sum_even_it(n, total = 0):
    if n == 0:
        return total
    return sum_even_it(n - 1, total + n);

print(sum_even_it(6))

21


c) We can of course express this in a declarative and Pythonic way, which is non-recursive. Write a function `sum_even_py` which returns the same result as above, but using comprehension or filter/map/reduce construct.

In [3]:
def sum_even_py(n):
    return sum([x for x in range(n + 1)])

print(sum_even_py(6))

21


## 2.2 Double/tree recursion

Sometimes we might find ourselves with branching structures, where there are several "smaller" cases to recurse over. This might for instance be the case when we have trees, lists-within-lists or the like.

[Note: In the tasks below, it might be helpful to gain an understanding of `isinstance`. See the documentation!]

a) One common use of recursion is to traverse recursive data structures. One exercise might be to _flatten_ nested lists or tuples. This is relatively simple with only one level of nesting, or when the structure follows a strict pattern, but for arbitrary nested sequences, a recursive approach is more natural. Implement a recursive function `myflatten` which can take an arbitrary structure of nested tuples and flattens it (in the sense of returning a new non-nested tuple with the same elements in the same order).

In [4]:
def myflatten(seq):
    
    if len(seq) == 0:
        return tuple()
    
    if isinstance(seq[0], tuple):
        result = myflatten(seq[0])
    else:
        result = (seq[0], )
    
    if len(seq[1:]) == 0:
        return result

    return result + myflatten(seq[1:])

def test_myflatten():
    tests = (
     ((), (), "the empty tuple"), 
     ((1,2,3), (1,2,3), "flat tuples invariant under flattening"), 
     ( (1, (2), 3, (4, 5, (6), 7), 8), (1, 2, 3, 4, 5, 6, 7, 8), "Arbitrarily nested tuples.")
    )
    for arg, expected_output, error_msg in tests:
        assert myflatten(arg) == expected_output, error_msg
        
test_myflatten()

In [5]:
print(myflatten(()))
print(myflatten((1,2,3)))
print(myflatten(((1, (2), 3, (4, 5, (6), 7), 8), (1, 2, 3, 4, 5, 6, 7, 8))))

()
(1, 2, 3)
(1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8)


b) Implement a function `exists_in(e, seq)` which returns `True` if the element `e` exists somewhere in the tuple `seq` (and `False` otherwise). `seq` might be nested and contain tuples-within-tuples.

In [6]:
def exists_in(e, seq):
    
    result = 0
    
    if (len(seq)) == 0:
        return 0
    
    if isinstance(seq[0], tuple):
        result = result + exists_in(e, seq[0])
    
    if (seq[0] == e):
        return 1
    elif len(seq) > 0:
        result = result + exists_in(e, seq[1:])
    
    return result > 0    

c) Write a few representative test cases, as in lab 2A.

In [7]:
def test_find_anywhere():
    tests = (
     ((1, (1, 2, 3)), (True), "First element not found."), 
     ((2, (1, 2, 3)), (True), "Intermediate element not found."), 
     ((8, (1, 2, 3)), (False), "Non existing value found."), 
     ((7, (7, (1, (2), 3, (4, 5, (6), 7), 8))), (True), "Arbitrarily nested tuple should be searchable as well."),
     ((9, (7, (1, (2), 3, (4, 5, (6), 7), 8))), (False), "Arbitrarily nested tuple should be searchable as well."))
    for arg, expected_output, error_msg in tests:
        assert exists_in(*arg) == expected_output, error_msg
        
test_find_anywhere()

d) One of the most famous recursive functions is the Quicksort function (https://en.wikipedia.org/wiki/Quicksort). It allows us to sort a sequence, with repeated values, in (amortized) log-linear time and with a logarithmic number of recursive calls. We will start by implementing Quicksort for a tuple of numbers.

Note that Wikipedia illustrates a more advanced _in-place_ version of Quicksort, with a more advanced partition function. For the purposes of this assignment, you can simply pass a new tuple or generator to each recursive call to quicksort. You may use eg _filter_ or a comprehension to create the inputs.

In [8]:
from random import sample, choice

# Write quicksort here:
def quicksort(seq):
    
    if isinstance(seq, int):
        return (seq, )
    
    if len(seq) == 0:
        return tuple()
    
    pivot = seq[0]
    seq_lower = tuple(filter(lambda x: x < pivot, seq))
    seq_upper = tuple(filter(lambda x: x > pivot, seq))
    
    return(quicksort(seq_lower) + (pivot, ) + quicksort(seq_upper))

a = tuple(sample(range(1000,2000), 1000))
b = quicksort(a)
a[0:10]

(1121, 1720, 1989, 1222, 1429, 1952, 1345, 1303, 1447, 1158)

In [9]:
b[0:10]

(1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009)

In [10]:
b[-10:]

(1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999)

## 3 Higher order functions (HOF)
 
A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, but we will call any function which returns a function or takes a function as an argument a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_)

In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's [$\lambda$-calculus](https://www.youtube.com/watch?v=eis11j_iGMs) which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

## 3.1 The three standard functions `map`, `reduce` and `filter`

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard built-in functions. Since Python 3, the `reduce` function needs to be imported from the `functools` module.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

A note from last year: usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by André Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

a) Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [11]:
from functools import reduce

def mysum(seq):
    return reduce(lambda x, y: x + y, seq)

mysum((4, 7, 1))

12

b) Implement a function `mylength` which uses `map` and `reduce` to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: Use `map` to convert the input to something which can easily be `reduce`:d.]

In [12]:
def mylength(seq):
    return mysum(map(lambda x: 1, seq))
    
print(mylength((4, 2, 5, 2, 5)))
print(mylength("test"))

5
4


## 3.2 Building your own higher order functions

a) Re-implement the three basic functional helper functions `map`, `filter` and `reduce` **as purely functional recursive functions**. You may not express this as eg comprehensions; the task is to practice figuring out this type of logic.

Note that the built-in versions of these functions work on multiple sequences of equal length if supplied, however, you can assume a single sequence as second parameter, i.e. you can also skip the third parameter to reduce.

In [13]:
seq = (1, 2, 3, 4, 5)
print(seq[1:])

(2, 3, 4, 5)


In [14]:
def mymap(f, seq):
    if (len(seq)) == 0:
        return tuple()
    return (f(seq[0]), ) + mymap(f, seq[1:])

mymap(lambda x:x**2, tuple(range(10)))

(0, 1, 4, 9, 16, 25, 36, 49, 64, 81)

In [15]:
def myfilter(f, seq):
    if (len(seq)) == 0:
        return tuple()
    if f(seq[0]):
        return (seq[0], ) + myfilter(f, seq[1:])
    return tuple() + myfilter(f, seq[1:])

myfilter(lambda x:x%2==0, tuple(range(10)))

(0, 2, 4, 6, 8)

You might note the similarities with how you implemented `sum_even`.

In [16]:
def myreduce(f, seq):
    if (len(seq)) == 2:
        return f(seq[0], seq[1])
    return f(seq[0], myreduce(f, seq[1:]))

myreduce(lambda x, y: x*y, tuple(range(1,5)))

24

## 3.3 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

a) Function composition is a common in both maths and programming. Write a function `compose` which takes two functions, $f$ and $g$, and produces the _composite_ function $f \circ g$, where $(f \circ g)(x) \Leftrightarrow f(g(x))$. Example use is given below.

In [17]:
from statistics import stdev, mean

def compose(f, g):
    return lambda x: f(g(x))

def myscale(vals):
    return [x/stdev(vals) for x in vals]

def myshift(vals):
    return [x-mean(vals) for x in vals]

standardize = compose(myscale, myshift)

print(standardize(range(-3, 8)))

[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


In [18]:
# To see how it works
a = range(-3, 8)
print(a)
a = myshift(a)
print(a)
a = myscale(a)
print(a)

range(-3, 8)
[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


b) Create a function `composition(*funs)` which takes a non-empty sequence of functions of one argument and returns their sequential composition. That is $composition(f_0,f_1, \ldots, f_n) = f_0 \circ f_1 \circ\ldots \circ f_n$. (The question of if $f\circ g \circ h$ should be read $f\circ (g\circ h)$ or $(f \circ g) \circ h$ is perfectly valid, but they turn out to be the same. That is, $\circ$ is associative.)

In [19]:
def composition(*funs):
    return reduce(lambda f, g: lambda x: f(g(x)), funs, lambda x: x)

standardize = composition(*[myscale, myshift])
print(standardize(range(-3, 8)))

[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


Hint: Don't remember what can be found in `*funs`? Print it! Don't know how the values should be combined? Write out some simple example on paper.

Note: This task demonstrates the generality of our constructs. Previously we worked with sequences of numbers and the like. Now we lift this to the level of working with functions as values, and instead of using combinators which work on numbers, we use function combinators in conjunction with our known patterns.

### Voluntary task: pipelining

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

You may want to use functions you have defined above.

from functools import reduce, partial
from math import sqrt 

def make_pipeline(*funs):
    pass
    return lambda vals: ???

# We can even drop the lambda vals : bit, using partial
# evaluation (see the help for functools.partial!)

def inc(x):
    return x+1

pipeline = make_pipeline(inc, abs, sqrt)

tuple(pipeline(range(-5,5)))

## 4. Simple declarative Pythonic patterns (involing HOF)

a) As preparation, create a named tuple type "coord" which has fields `x` and `y`.

In [20]:
from collections import namedtuple
coord = namedtuple('coord', 'x y')

five_three = coord(5,3)
assert five_three.x == 5, "first element is the x coordinate"
assert five_three.y == 3, "the second element is the y coordinate"

b) Generate a $10^7$ random coordinates, with $x$ and $y$ coordinates drawn uniformly from [-1000,1000]. Save the tuple of those with $x + y > 0$ as `rnd_coords`. How many are there?

In [21]:
from random import uniform

rnd_coords = [None]*10_000_000 # Tupel won't work here!
rnd_coords = map(lambda x: coord(uniform(-1000, 1000), uniform(-1000, 1000)), rnd_coords)
rnd_coords = tuple(filter(lambda point: point.x + point.y > 0, rnd_coords))

print(len(rnd_coords))
rnd_coords[0:10]

4998998


(coord(x=-129.80403921240827, y=527.6372343781306),
 coord(x=956.6651115469208, y=934.3596327828768),
 coord(x=823.7680008759442, y=53.141043374457695),
 coord(x=116.68204389382367, y=193.98071354114109),
 coord(x=-770.936389232472, y=822.5894188585332),
 coord(x=988.213705212087, y=-612.062408295779),
 coord(x=-6.372145858167414, y=71.76555397881043),
 coord(x=174.8676750877646, y=606.5773139526309),
 coord(x=712.5027868734462, y=-488.44262809232816),
 coord(x=209.96006752845187, y=339.73060531512306))

[Note: If this takes a while, you might want to consider when the elements are generated and saved.]

**Before having solved the tasks below, consider setting `coords` to a smaller set (eg generate $10^3$ elements instead of $10^7$ to start with).**

c) Let `sorted_rnd` be the coordinated sorted first by the `x` component and then the `y`. Use a built-in Python sorting function. Do you need any extra parameters? Why? Why not? How would you find out where the order comes from (and might it be consistent but useless, eg sorting the elements by memory location)?

**Answer:** As the instructions are unclear and we don't really now how we are supposed to sort, we did it in three ways:
- First order by X and if X is the same, then by Y. This will mostly not happen with floating point numbers.
- We sort the set of all X and all Y and then match them again.
- First sort by X, then sort by Y.

We're not sure what is meant by extra parameters, as we simply sort by a key, so maybe this is ment as a parameter. If this was not meant then no, we didn't use any parameters.

The last question, where the order comes from, is unclear to us. What is meant by that? Do you mean from which "implementation" is coming from? We would look in the Python documentation.

We no these answers might not be as wished, but we and others groups couldn't figure out what was meant here, sorry.

In [22]:
# Sorting by X, if matching sort by Y
sorted_rnd = sorted(rnd_coords, key = lambda point: (point.x, point.y))
sorted_rnd[0:10]

[coord(x=-999.6023073711764, y=999.8777468492854),
 coord(x=-999.3630677377996, y=999.8170731858365),
 coord(x=-999.2597173446416, y=999.6052932305633),
 coord(x=-998.5851200781603, y=998.7538382497073),
 coord(x=-997.8393674435251, y=998.7048632006515),
 coord(x=-997.5810661837587, y=998.0805540417305),
 coord(x=-997.5786495288943, y=998.7982255512293),
 coord(x=-997.3341556956741, y=997.6384172819239),
 coord(x=-997.249023177508, y=998.1764290943643),
 coord(x=-997.2469857223532, y=997.3296132870537)]

In [23]:
sorted_rnd[-10:]

[coord(x=999.9978643104605, y=-298.737988093035),
 coord(x=999.9982422339713, y=723.8979548666507),
 coord(x=999.9984769134721, y=202.55111076719754),
 coord(x=999.9989104224908, y=715.1122968361071),
 coord(x=999.9992033477463, y=-490.21559253075964),
 coord(x=999.9994928229346, y=-497.6014267789346),
 coord(x=999.9996742642661, y=96.3974034242674),
 coord(x=999.9997393642291, y=-236.53892183984772),
 coord(x=999.9997851388625, y=-339.7899333735637),
 coord(x=999.9997907698767, y=335.54544822021853)]

In [24]:
# Sorting X and Y independently
sorted_rnd_x = sorted(rnd_coords, key = lambda point: (point.x))
sorted_rnd_y = sorted(rnd_coords, key = lambda point: (point.y))
sorted_rnd = tuple(map(lambda point: coord(point[0].x, point[1].y), zip(sorted_rnd_x, sorted_rnd_y)))
sorted_rnd[0:10]

(coord(x=-999.6023073711764, y=-999.4725691904531),
 coord(x=-999.3630677377996, y=-999.3569407949241),
 coord(x=-999.2597173446416, y=-998.0626283315943),
 coord(x=-998.5851200781603, y=-997.7148550626489),
 coord(x=-997.8393674435251, y=-997.6428685114316),
 coord(x=-997.5810661837587, y=-997.0831565295764),
 coord(x=-997.5786495288943, y=-996.7749396449244),
 coord(x=-997.3341556956741, y=-996.6744186105891),
 coord(x=-997.249023177508, y=-996.6335348942416),
 coord(x=-997.2469857223532, y=-996.6132675965566))

In [25]:
sorted_rnd[-10:]

(coord(x=999.9978643104605, y=999.9976745615668),
 coord(x=999.9982422339713, y=999.9977425451837),
 coord(x=999.9984769134721, y=999.9977795531779),
 coord(x=999.9989104224908, y=999.9980923021017),
 coord(x=999.9992033477463, y=999.9983447770417),
 coord(x=999.9994928229346, y=999.9985544601359),
 coord(x=999.9996742642661, y=999.998584848709),
 coord(x=999.9997393642291, y=999.9988473546282),
 coord(x=999.9997851388625, y=999.9994788761121),
 coord(x=999.9997907698767, y=999.9996077903784))

In [26]:
# First sorting by X, then by Y
sorted_rnd = sorted(sorted(rnd_coords, key = lambda point: (point.x)), key = lambda point: (point.x))
sorted_rnd[0:10]

[coord(x=-999.6023073711764, y=999.8777468492854),
 coord(x=-999.3630677377996, y=999.8170731858365),
 coord(x=-999.2597173446416, y=999.6052932305633),
 coord(x=-998.5851200781603, y=998.7538382497073),
 coord(x=-997.8393674435251, y=998.7048632006515),
 coord(x=-997.5810661837587, y=998.0805540417305),
 coord(x=-997.5786495288943, y=998.7982255512293),
 coord(x=-997.3341556956741, y=997.6384172819239),
 coord(x=-997.249023177508, y=998.1764290943643),
 coord(x=-997.2469857223532, y=997.3296132870537)]

In [27]:
sorted_rnd[-10:]

[coord(x=999.9978643104605, y=-298.737988093035),
 coord(x=999.9982422339713, y=723.8979548666507),
 coord(x=999.9984769134721, y=202.55111076719754),
 coord(x=999.9989104224908, y=715.1122968361071),
 coord(x=999.9992033477463, y=-490.21559253075964),
 coord(x=999.9994928229346, y=-497.6014267789346),
 coord(x=999.9996742642661, y=96.3974034242674),
 coord(x=999.9997393642291, y=-236.53892183984772),
 coord(x=999.9997851388625, y=-339.7899333735637),
 coord(x=999.9997907698767, y=335.54544822021853)]

[General words of advice:

* During testing, you might want to use a smaller data set (and then try it out at a larger set).
* You might not want to display the entire list to see if you're right all the time. Slicing out the first and last elements, say the first or last 10, might provide some hints.
* You could naturally define a function which checks that the list is in order (or performs some probabilistic sampling test), to test this.]

d) Sort the values (in the sense of returning a new sorted tuple) by their Euclidean distance to the point (5,3). Continue using a built-in Python sorting function.

In [28]:
import math
pts_near_53 = sorted(rnd_coords, key = lambda point : math.sqrt((point.x - 5)**2 + (point.y - 3)**2))
pts_near_53[0:10]

[coord(x=5.388416965218084, y=3.0222128186163673),
 coord(x=4.547841484095329, y=3.1023225042243894),
 coord(x=4.663114974437121, y=2.326212495794607),
 coord(x=5.642371346125515, y=3.614708408108413),
 coord(x=5.23344366861545, y=3.915112227136092),
 coord(x=5.890776342998038, y=2.5705858555404575),
 coord(x=5.977948911184171, y=2.79121135987225),
 coord(x=4.532140159446271, y=3.917055767312945),
 coord(x=4.077778591714377, y=2.402533016072198),
 coord(x=3.896855925946511, y=2.9136308544843814)]

In [29]:
pts_near_53[-10:]

[coord(x=-996.1640922561165, y=999.5341001919514),
 coord(x=-996.5793486250502, y=999.3483218994611),
 coord(x=-996.2504799517134, y=999.922959615619),
 coord(x=-997.5786495288943, y=998.7982255512293),
 coord(x=-996.5006494439466, y=999.9675781459891),
 coord(x=-997.8393674435251, y=998.7048632006515),
 coord(x=-998.5851200781603, y=998.7538382497073),
 coord(x=-999.2597173446416, y=999.6052932305633),
 coord(x=-999.3630677377996, y=999.8170731858365),
 coord(x=-999.6023073711764, y=999.8777468492854)]

Note: here we customise the behaviour of a built-in function by passing it information about our intended ordering.

d) Define the function `sorted_by_distance(origo)` which takes a coordinate `origo` and returns a function which sorts the sequence by the euclidean distance to `origo`. (Ie those closest to origo come first in the list.)

In [30]:
def sorted_by_distance(origo):
    return lambda seq : sorted(rnd_coords, key = lambda point : math.sqrt((point.x - origo.x)**2 + (point.y - origo.y)**2))


ordered_by_closeness_to_53 = sorted_by_distance(coord(5,3))   # Return the function.
pts_near_53_2 = ordered_by_closeness_to_53(rnd_coords)     # Applying the function 

assert pts_near_53 == pts_near_53_2

In [31]:
pts_near_53[0:10]

[coord(x=5.388416965218084, y=3.0222128186163673),
 coord(x=4.547841484095329, y=3.1023225042243894),
 coord(x=4.663114974437121, y=2.326212495794607),
 coord(x=5.642371346125515, y=3.614708408108413),
 coord(x=5.23344366861545, y=3.915112227136092),
 coord(x=5.890776342998038, y=2.5705858555404575),
 coord(x=5.977948911184171, y=2.79121135987225),
 coord(x=4.532140159446271, y=3.917055767312945),
 coord(x=4.077778591714377, y=2.402533016072198),
 coord(x=3.896855925946511, y=2.9136308544843814)]

In [32]:
pts_near_53_2[0:10]

[coord(x=5.388416965218084, y=3.0222128186163673),
 coord(x=4.547841484095329, y=3.1023225042243894),
 coord(x=4.663114974437121, y=2.326212495794607),
 coord(x=5.642371346125515, y=3.614708408108413),
 coord(x=5.23344366861545, y=3.915112227136092),
 coord(x=5.890776342998038, y=2.5705858555404575),
 coord(x=5.977948911184171, y=2.79121135987225),
 coord(x=4.532140159446271, y=3.917055767312945),
 coord(x=4.077778591714377, y=2.402533016072198),
 coord(x=3.896855925946511, y=2.9136308544843814)]

[Note: Here we extend the work above to a higher-order function, which uses the local value of `origo`. In essence, this task summarises higher order functionality - we create a closure, return a function and use a custom ordering ]

e) So far in the course, we have seen, and possibly used `enumerate`, `range`, `zip`, `map` and `filter` as declarative constructs (along with the general comprehension syntax). Now we introduce a further useful iterator construct. Construct something called `reverse_squared` which when prompted would give us the squares of elements 0,...,N _but in reverse_ (that is $N^2, (N-1)^2, ..., 2^2, 1^2, 0^2$).

In [33]:
# The time it takes to run this shouldn't really depend on if you use SMALL_N or BIG_N.

BIG_N = 99999999
SMALL_N = 999

N = BIG_N    # change this to test later on

reverse_squares = map(lambda value: value**2, range(N, -1, -1)) # your code here

In [34]:
# Experimentation: copy and paste your code from above into this cell.
# This is rather crude, but we want you to to be able to trust that any
# slowness in the cell above can be found by reference to that code, not the
# profiling code below.

# We already pasted it below, so we have no idea why we should paste it here as well.
# We know it's simply a generator and values are calcualted on the fly when needed, so we have no trust issues here... :)

import profile

# We cut and paste this code 
BIG_N = 99999999
SMALL_N = 999

N = BIG_N    
# Look at the run time. Switching from BIG_N to SMALL_N shouldn't really matter.
# This suggests that we have quick access to elements at the end of our (squared) range.

reverse_squares = map(lambda value: value**2, range(N, -1, -1)) # <<----------- Your code from the cell above goes here.


profile.run("print('Did we find it? ', {} in reverse_squares)".format( N**2 ))

Did we find it?  True
         66 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 :0(acquire)
        5    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(exec)
        4    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        1    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 <ipython-input-34-6d4c57872b8f>:19(<lambda>)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:307(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:320(_schedule_flush)
        4    0.000    0.000    0.00

Note: once you know of the construct, this task is extremely simple. It mostly serves as a demonstration of the availability of these constructs, and how they can be combined. Also, it points to efficiency considerations when using declarative iterator constructs as opposed to fixed computed structures.

As the profiling code above suggests, where we redefine the object in every run, we do not have a purely functional construct. In that case, we wouldn't be able to exhaust the values.

[Additional reading: some additional tools are available in the `itertools` module.]

## 5 Mutating function state

A function always has access to the environment in which it was created. Usually, this means that the function can access global variables. It also means that it can access and modify local bindings from where it was created.

A closure is a function which has access to an environment which is not accessible from outside the function (but which is not destroyed when the function returns). I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

a) Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [35]:
def make_counter(n):
    storage = n
    
    def increase():
        nonlocal storage
        storage = storage + 1
        return storage
    
    return increase

counter_A = make_counter(0)
counter_B = make_counter(15)
print("To show that the functions do not affect each others' states, consider the printout:")
print("counter_A returns: {}".format(counter_A()))
print("counter_A returns: {}".format(counter_A()))
print("counter_B returns: {}".format(counter_B()))
print("counter_A returns: {} (was it affected by the call to counter_B?)".format(counter_A()))

To show that the functions do not affect each others' states, consider the printout:
counter_A returns: 1
counter_A returns: 2
counter_B returns: 16
counter_A returns: 3 (was it affected by the call to counter_B?)


## 6. Use case: parametrisation

Above, we see how `sorted` can be parametrised with information about the intended order. We want to extend our `quicksort` to work the same way. We should be able to provide the way for it to tell if object A in the tuple should come before object B, or after. This is done by mapping the objects onto something where we do have an order.

a) Copy your code from the `quicksort` task above, and extend it. Call the function `quicksort_param` for parametrised, and allow a key parameter to be passed in (like in `sorted`). Note that the key function should be optional. We thus want default arguments.

In [36]:
from random import random

# Write quicksort_param here:

def quicksort_param(seq, key = lambda value: value):
    
    if isinstance(tuple(map(key, seq)), int):
        return (seq, )
    
    if len(tuple(map(key, seq))) == 0:
        return tuple()
    
    pivot = key(seq[0])
    
    seq_lower = tuple(filter(lambda x: key(x) < pivot, seq))
    seq_upper = tuple(filter(lambda x: key(x) > pivot, seq))
    
    return(quicksort_param(seq_lower, key) + (seq[0], ) + quicksort_param(seq_upper, key))

a = tuple(tuple(random() for i in range(3)) for j in range(10))
b = quicksort_param(a, sum)   # Elements are three-tuples. Those with smallest sums of values should come first.
a

((0.22212408814806162, 0.20537639964183985, 0.7793208895773884),
 (0.13609762793474522, 0.9937457091018332, 0.6013814416963916),
 (0.6486771517963844, 0.31049116272786115, 0.03518220966281571),
 (0.28136419609808805, 0.07554085491187112, 0.6304701082816673),
 (0.7677493519332991, 0.20019636317863954, 0.06275640430082641),
 (0.7985942305911453, 0.9761296046964922, 0.8429027624750294),
 (0.1324138228043349, 0.42839031047124254, 0.4496820833364502),
 (0.2782714100458632, 0.7226672196378288, 0.3349274555744536),
 (0.8097874835519704, 0.8530673132854537, 0.8329616400784645),
 (0.28829717195036253, 0.38785175470192623, 0.28002773712899787))

In [37]:
b

((0.28829717195036253, 0.38785175470192623, 0.28002773712899787),
 (0.28136419609808805, 0.07554085491187112, 0.6304701082816673),
 (0.6486771517963844, 0.31049116272786115, 0.03518220966281571),
 (0.1324138228043349, 0.42839031047124254, 0.4496820833364502),
 (0.7677493519332991, 0.20019636317863954, 0.06275640430082641),
 (0.22212408814806162, 0.20537639964183985, 0.7793208895773884),
 (0.2782714100458632, 0.7226672196378288, 0.3349274555744536),
 (0.13609762793474522, 0.9937457091018332, 0.6013814416963916),
 (0.8097874835519704, 0.8530673132854537, 0.8329616400784645),
 (0.7985942305911453, 0.9761296046964922, 0.8429027624750294))

In [38]:
tuple(map(sum, a))

(1.20682137736729,
 1.73122477873297,
 0.9943505241870613,
 0.9873751592916264,
 1.030702119412765,
 2.617626597762667,
 1.0104862166120276,
 1.3358660852581457,
 2.495816436915889,
 0.9561766637812866)

In [39]:
tuple(map(sum, b))

(0.9561766637812866,
 0.9873751592916264,
 0.9943505241870613,
 1.0104862166120276,
 1.030702119412765,
 1.20682137736729,
 1.3358660852581457,
 1.73122477873297,
 2.495816436915889,
 2.617626597762667)

In [40]:
quicksort_param([5,2,9,200])

(2, 5, 9, 200)

## Attribution

Lab by Johan Falkenjack (2018), extended and rewritten by Anders Märak Leffler (2019).

License [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)